# PREVIOUS ACTIVITY !

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

# ==========================================
# 👇 INSTRUCTOR CONFIGURATION (DO NOT EDIT)
# ==========================================
WEBHOOK_URL = "https://kartik987.app.n8n.cloud/webhook/faceoff"

def submit_work(df, student_name):
    """
    Submits the current DataFrame to the AI Auto-Grader.

    Args:
        df (pd.DataFrame): The dataframe containing your work.
        student_name (str): Your unique name or ID.
    """
    try:
        # 1. Create a clean copy to avoid modifying the student's actual dataframe
        #    We replace Infinity values which crash JSON conversion
        df_clean = df.copy()
        df_clean = df_clean.replace([np.inf, -np.inf], np.nan)

        # 2. Convert NaN to None (which becomes valid JSON 'null')
        #    We cast to object to ensure integers don't error out when assigned None
        df_clean = df_clean.astype(object).where(pd.notnull(df_clean), None)

        # 3. Prepare the Payload
        payload = {
            "student_id": str(student_name),
            "data": df_clean.to_dict(orient='records'),
            "columns": list(df.columns)
        }

        print(f"📡 Sending data for {student_name}...")

        # 4. Send to the AI Agent
        response = requests.post(WEBHOOK_URL, json=payload)

        # 5. Display Feedback
        if response.status_code == 200:
            res = response.json()
            score = res.get('score', 0)
            feedback = res.get('feedback', 'No feedback provided.')

            print(f"\n{'='*40}")
            print(f"🏆 SCORE: {score}%")
            print(f"📝 FEEDBACK: {feedback}")
            print(f"{'='*40}\n")
        else:
            print(f"⚠️ Server Error: {response.status_code}")
            print("Check your internet connection or ask the instructor.")

    except Exception as e:
        print(f"❌ Submission Failed: {str(e)}")
        print("Tip: Make sure your DataFrame is not empty and contains valid data.")

# ==========================================
# 👆 END OF SETUP CODE
# ==========================================

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load data
df = pd.read_csv('(12)Data_Face_Off.csv')
df.columns = df.columns.str.strip()

# ===============================
# 1. FILL MISSING VALUES
# ===============================
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Performance_Score'] = df['Performance_Score'].fillna(df['Performance_Score'].mean())
df['Salary_LPA'] = df['Salary_LPA'].fillna(df['Salary_LPA'].median())

# ===============================
# 2. IQR-BASED ROW REMOVAL (CRITICAL)
# ===============================
perf_upper = df['Performance_Score'].quantile(0.95)
df.loc[df['Performance_Score'] > perf_upper, 'Performance_Score'] = perf_upper

num_cols = ['Age', 'Weekly_Hours', 'Salary_LPA']

df_clean = df.copy()

for col in num_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df_clean = df_clean[
        (df_clean[col] >= lower) & (df_clean[col] <= upper)
    ]


# ===============================
# 3. ONE-HOT ENCODE TEXT COLUMNS
# ===============================
cat_cols = df_clean.select_dtypes(include='object').columns
df_clean = pd.get_dummies(df_clean, columns=cat_cols, drop_first=False)

# ===============================
# 4. MIN-MAX SCALING
# ===============================
scaler = MinMaxScaler()
df_clean[num_cols] = scaler.fit_transform(df_clean[num_cols])

# ===============================
# 5. SUBMIT
# ===============================
submit_work(df_clean, "Tejaswini")

📡 Sending data for Tejaswini...

🏆 SCORE: 80%
📝 FEEDBACK: ✅ Hours Outlier removed | ✅ Perf Outlier capped | ✅ Data Scaled | ✅ Age Filled (Scaled) | ✅ Salary Filled (Scaled) | ⚠️ Perf Value Deviation | ❌ Education Col Missing | ✅ One-Hot Encoded



# KAGGLE CONTEST -> DECISION TREE THE DILEMMA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier , plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [ ]:
test_df.head()

,id,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,...,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44
0,3500,NaN,1.009302,7.536494,-2.481251,0.811764,-0.787350,-1.941184,-2.100791,-0.659139,...,6.199141,3.061799,-0.570998,-0.444660,13.412987,1.514965,-3.505029,2.293413,-2.699701,-5.926544
1,3501,-6.317381,NaN,-1.300082,NaN,1.817092,0.993657,-1.140717,-1.641167,0.207558,...,1.773402,-1.210002,0.038987,0.528341,-0.385481,-0.962171,3.371620,-1.328556,1.367022,-0.084907
2,3502,NaN,-1.515210,-0.804519,0.984809,0.784271,NaN,2.371451,-0.276326,0.962838,...,-4.784820,2.083148,4.569049,2.856206,1.401604,-4.944263,-1.170930,-0.505483,-3.265877,-0.333070
3,3503,0.221118,0.879124,2.929880,5.727252,2.090450,-0.789070,NaN,-1.414831,-0.230936,...,2.294427,-2.161243,-4.621357,0.984736,-5.824983,0.524878,-1.582758,-2.682567,-6.422341,-2.631306
4,3504,-4.078059,1.913713,-0.085280,3.626643,8.010738,NaN,0.260104,2.343076,-0.501840,...,-3.736430,-4.094246,-4.852491,-0.950690,9.172851,0.330698,-5.436552,-1.657203,-3.137068,0.002860


In [ ]:
train_df.head()

,id,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,...,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,target
0,0,0.700663,0.544727,-1.507995,4.600504,NaN,NaN,1.317958,-2.184807,-0.536969,...,-1.720922,-1.312491,-0.240285,-9.176276,-0.964167,-2.819809,2.038961,-6.405272,-1.549836,0
1,1,7.065902,-0.538209,0.122591,4.501849,3.746408,-1.440206,2.971289,-5.086634,0.223375,...,-1.573417,-4.997023,-0.200928,-13.964100,0.137237,0.949989,3.166724,-2.882468,-4.629712,0
2,2,-6.520469,0.154616,-1.491436,-1.485808,5.691309,2.038025,NaN,NaN,-1.049342,...,-1.257423,-4.822252,-0.881965,1.771664,3.982637,3.988630,0.281435,-4.207008,2.545696,1
3,3,1.865736,NaN,0.706182,5.158291,-2.756292,NaN,NaN,3.835877,-0.309256,...,-1.226800,1.701869,0.918699,18.469356,-3.258649,-1.517424,-5.783032,-2.278890,1.300029,1
4,4,NaN,-0.258951,-0.962966,2.122233,1.378614,-1.218209,5.401169,-0.199775,0.510567,...,0.715215,3.969463,-0.686476,-7.734691,1.162694,0.891141,0.100865,-2.671053,-2.079749,0


In [ ]:
train_df.isnull().sum()

,0
id,0
feature_00,533
feature_01,572
feature_02,505
feature_03,517
feature_04,516
feature_05,514
feature_06,507
feature_07,545
feature_08,507


In [ ]:
# Dropping unrequired columns from train_df
X = train_df.drop(columns=["id", "target"])
y = train_df["target"]

X_test = test_df.drop(columns=["id"])

In [ ]:
# Filling missing values
median_val = X.median()

# fill missing values
X = X.fillna(median_val)
X_test = X_test.fillna(median_val)

In [ ]:
# Handle outliers
for col in X.columns:
    low  = X[col].quantile(0.01)
    high = X[col].quantile(0.99)

    X[col] = X[col].clip(low, high)
    X_test[col] = X_test[col].clip(low, high)

In [ ]:
# Splitting train test data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# DecisionTreeClassifier model training
model = DecisionTreeClassifier(max_depth=7,min_samples_leaf=15,random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=15, random_state=42)

In [ ]:
# Checking accuracy score
y_preds = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_preds))

Accuracy: 0.7428571428571429


In [ ]:
# Submission
y_test_preds = model.predict(X_test)
submission = pd.DataFrame({
    "id": test_df["id"],
    "target": y_test_preds
})

submission.to_csv("submission.csv", index=False)

In [ ]:
# KAGGLE CONTEST -> DECISION TREE THE DILEMMA

import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# -----------------------------
# Load Data
# -----------------------------
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

# -----------------------------
# Separate Features & Target
# -----------------------------
X = train_df.drop(columns=["id", "target"])
y = train_df["target"]

X_test = test_df.drop(columns=["id"])

# -----------------------------
# Handle Missing Values (TRAIN-ONLY logic)
# -----------------------------
median_vals = X.median()

X = X.fillna(median_vals)
X_test = X_test.fillna(median_vals)

# -----------------------------
# Handle Outliers (Percentile Capping)
# -----------------------------
for col in X.columns:
    low  = X[col].quantile(0.02)
    high = X[col].quantile(0.98)

    X[col] = X[col].clip(low, high)
    X_test[col] = X_test[col].clip(low, high)

# -----------------------------
# Train–Validation Split
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# Decision Tree Model
# -----------------------------
model = DecisionTreeClassifier(
    max_depth=7,
    min_samples_leaf=15,
    random_state=42
)

model.fit(X_train, y_train)

# -----------------------------
# Validation Accuracy
# -----------------------------
y_val_preds = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_val_preds))

# -----------------------------
# Train on FULL data & Predict Test
# -----------------------------
model.fit(X, y)
y_test_preds = model.predict(X_test)

# -----------------------------
# Submission File
# -----------------------------
submission = pd.DataFrame({
    "id": test_df["id"],
    "target": y_test_preds
})

submission.to_csv("submission.csv", index=False)


Accuracy: 0.7428571428571429


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split training data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train Decision Tree
model = DecisionTreeClassifier(
    max_depth=7,
    min_samples_leaf=15,  # increase to prevent tiny leaves
    random_state=42
)
model.fit(X_train, y_train)

# Accuracy on training data
train_acc = accuracy_score(y_train, model.predict(X_train))
print("Training Accuracy:", round(train_acc, 4))

# Accuracy on validation data
val_acc = accuracy_score(y_val, model.predict(X_val))
print("Validation Accuracy:", round(val_acc, 4))

# Optional: simple overfitting check
if train_acc - val_acc > 0.05:
    print("Warning: Model may be overfitting")
else:
    print("No significant overfitting detected")

Training Accuracy: 0.8143
Validation Accuracy: 0.7429
